In [2]:
import pandas as pd
import numpy as np

In [8]:
data_path = "C:/Users/armcl/Python/Springboard_Cap1/Data/"

In [17]:
#Read Redfin data
redfin = pd.read_csv(data_path + "Redfin/Santa_Clara_County.csv")
#Remove irrelvant and poor quality (for example 'DAYS ON MARKET') data columns
redfin = redfin.drop(columns = ['SALE TYPE', 'DAYS ON MARKET', 'STATUS', 'NEXT OPEN HOUSE START TIME', 
                       'NEXT OPEN HOUSE END TIME', 'SOURCE', 'FAVORITE', 'INTERESTED', 
                       'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)'])

redfin.head()

,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,MLS#,LATITUDE,LONGITUDE
0,July-2-2019,Single Family Residential,788 Orkney Ave,SANTA CLARA,CA,95054,1050000.0,3.0,2.0,SANTA CLARA,1097.0,6242.0,1961.0,957.0,NaN,ML81756181,37.389165,-121.947778
1,April-19-2018,Condo/Co-op,3901 Lick Mill Blvd #259,SANTA CLARA,CA,95054,882000.0,1.0,1.0,SANTA CLARA,865.0,865.0,2005.0,1020.0,383.0,ML81696133,37.396173,-121.943140
2,October-20-2017,Condo/Co-op,3901 Lick Mill Blvd #166,SANTA CLARA,CA,95054,680000.0,1.0,1.0,SANTA CLARA,900.0,900.0,2005.0,756.0,390.0,ML81679952,37.396330,-121.943234
3,October-5-2018,Single Family Residential,956 Baird Ave,SANTA CLARA,CA,95054,1182375.0,4.0,2.0,SANTA CLARA,1466.0,7618.0,1961.0,807.0,NaN,ML81721796,37.387366,-121.950200
4,May-29-2018,Single Family Residential,3705 Eastwood Cir,SANTA CLARA,CA,95054,1570000.0,4.0,2.0,SANTA CLARA,1666.0,6000.0,1969.0,942.0,NaN,ML81703477,37.392009,-121.940445


In [18]:
#Seems only Property Type, State, Latitude and Longitude have non NAs
redfin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29919 entries, 0 to 29918
Data columns (total 18 columns):
SOLD DATE             25996 non-null object
PROPERTY TYPE         29919 non-null object
ADDRESS               29800 non-null object
CITY                  29899 non-null object
STATE OR PROVINCE     29919 non-null object
ZIP OR POSTAL CODE    29902 non-null object
PRICE                 29789 non-null float64
BEDS                  29465 non-null float64
BATHS                 28878 non-null float64
LOCATION              25985 non-null object
SQUARE FEET           29768 non-null float64
LOT SIZE              28491 non-null float64
YEAR BUILT            29747 non-null float64
$/SQUARE FEET         29640 non-null float64
HOA/MONTH             9695 non-null float64
MLS#                  25996 non-null object
LATITUDE              29919 non-null float64
LONGITUDE             29919 non-null float64
dtypes: float64(10), object(8)
memory usage: 4.1+ MB


In [19]:
redfin.describe()
#Observations of data issues
#Min and Max suggests outliers, need further investigations

#There are duplicate data while downloading when setting filters, need to be removed

#For type multifamily (5+ unit) but beds < 5 or missing, should be removed
#Should we only focus on type SFH, Townhouse, and Condo?

#There are observations with state in PA or TX, should be removed

#Sorted by price, some observations with super high prices are apartments/office/land and should be removed
#Observations with prices missing or too low are bad data, need to be removed

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
count,2.978900e+04,29465.000000,28878.000000,29768.000000,2.849100e+04,29747.000000,2.964000e+04,9695.000000,29919.000000,29919.000000
mean,1.452307e+06,3.290582,2.341246,1888.215298,1.013576e+04,1974.448516,8.681001e+02,324.167406,37.299072,-121.898079
std,2.739201e+06,1.090651,0.846894,3846.153051,4.236792e+04,23.265612,1.063604e+04,203.997016,0.093041,0.127440
min,3.895000e+03,0.000000,0.500000,0.000000,1.000000e+00,1858.000000,3.000000e+00,1.000000,36.964850,-122.180281
25%,8.500000e+05,3.000000,2.000000,1234.000000,3.060000e+03,1959.000000,5.630000e+02,208.000000,37.257107,-121.982254
50%,1.155000e+06,3.000000,2.000000,1593.500000,6.103000e+03,1972.000000,7.195000e+02,325.000000,37.310587,-121.900109
75%,1.600000e+06,4.000000,3.000000,2106.000000,8.001500e+03,1990.000000,9.420000e+02,412.000000,37.362173,-121.834079
max,2.555000e+08,12.000000,10.000000,350451.000000,2.652368e+06,2019.000000,1.730000e+06,6000.000000,37.465350,-121.419033


In [31]:
#Read Airbnb data
airbnb = pd.read_csv(data_path + "Airbnb/Santa Clara/listings.csv")

#Keep the relevant columns
airbnb = airbnb[['id', 'last_scraped', 'host_since', 'host_response_time', 'host_response_rate', 'host_is_superhost',
                'host_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood',
                'neighbourhood_cleansed', 'city', 'state', 'zipcode', 'latitude', 'longitude', 'property_type',
                'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
                'price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'guests_included',
                'extra_people', 'minimum_nights', 'maximum_nights', 'calendar_updated', 'availability_30',
                'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
                'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value',
                'instant_bookable', 'cancellation_policy', 'require_guest_profile_picture', 'require_guest_phone_verification',
                'calculated_host_listings_count', 'reviews_per_month']]

airbnb.head()

,id,last_scraped,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,street,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,4952,2019-07-09,2009-01-24,within an hour,100%,t,5.0,t,f,"Palo Alto, CA, United States",...,10.0,10.0,10.0,10.0,f,moderate,f,f,5,0.62
1,11464,2019-07-09,2009-09-30,within a few hours,100%,f,14.0,t,f,"Santa Clara, CA, United States",...,10.0,9.0,9.0,9.0,f,moderate,f,t,13,0.08
2,21373,2019-07-09,2009-01-24,within an hour,100%,t,5.0,t,f,"Palo Alto, CA, United States",...,10.0,10.0,10.0,10.0,f,moderate,f,f,5,2.08
3,23732,2019-07-09,2008-08-15,NaN,NaN,f,1.0,t,t,"Mountain View, CA, United States",...,10.0,10.0,10.0,10.0,f,moderate,f,f,1,0.03
4,29790,2019-07-09,2010-05-19,within an hour,100%,f,1.0,t,t,"Mountain View, CA, United States",...,10.0,10.0,10.0,9.0,f,flexible,f,f,1,0.90


In [32]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7196 entries, 0 to 7195
Data columns (total 57 columns):
id                                  7196 non-null int64
last_scraped                        7196 non-null object
host_since                          7194 non-null object
host_response_time                  6463 non-null object
host_response_rate                  6463 non-null object
host_is_superhost                   7194 non-null object
host_listings_count                 7194 non-null float64
host_has_profile_pic                7194 non-null object
host_identity_verified              7194 non-null object
street                              7196 non-null object
neighbourhood                       6718 non-null object
neighbourhood_cleansed              7196 non-null object
city                                7187 non-null object
state                               7196 non-null object
zipcode                             6814 non-null object
latitude                            71

In [33]:
airbnb.describe()

,id,host_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,guests_included,...,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month
count,7.196000e+03,7194.000000,7196.000000,7196.000000,7196.000000,7195.000000,7195.000000,7190.000000,29.000000,7196.000000,...,7196.000000,5858.000000,5857.000000,5857.000000,5855.000000,5856.000000,5856.000000,5855.000000,7196.000000,5906.000000
mean,2.192405e+07,85.044760,37.354823,-121.973816,3.215258,1.410215,1.456150,1.907789,813.310345,1.708310,...,12.420650,95.270058,9.721188,9.578965,9.812810,9.806694,9.770833,9.537660,26.061145,1.743493
std,1.027908e+07,282.907513,0.064165,0.108431,2.442318,0.719481,1.031403,1.555120,621.796769,1.611665,...,19.070046,7.517582,0.759019,0.827892,0.661659,0.656723,0.605477,0.798385,73.800338,1.839373
min,4.952000e+03,0.000000,36.965600,-122.197880,1.000000,0.000000,0.000000,0.000000,100.000000,1.000000,...,0.000000,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.010000
25%,1.362560e+07,1.000000,37.321625,-122.063413,2.000000,1.000000,1.000000,1.000000,225.000000,1.000000,...,1.000000,94.000000,10.000000,9.000000,10.000000,10.000000,10.000000,9.000000,1.000000,0.480000
50%,2.360020e+07,3.000000,37.363435,-121.968270,2.000000,1.000000,1.000000,1.000000,800.000000,1.000000,...,5.000000,97.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,3.000000,1.080000
75%,3.072629e+07,13.000000,37.399932,-121.892507,4.000000,2.000000,2.000000,2.000000,1300.000000,2.000000,...,16.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,2.370000
max,3.653630e+07,1295.000000,37.462950,-121.380120,16.000000,8.000000,10.000000,26.000000,2500.000000,16.000000,...,174.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,344.000000,15.430000
